# Merge data for Panama Papers

In [1]:
import pandas as pd
import numpy as np

### Read in files

In [59]:
edges = pd.read_csv('panama/panama_papers.edges.csv')
entities = pd.read_csv('panama/panama_papers.nodes.entity.csv')
officers = pd.read_csv("panama/panama_papers.nodes.officer.csv")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Merge on 'node_id' from edges

TO DO: investigate 'intermediary'

In [3]:
edges_merged = pd.merge(edges[edges['rel_type'] == 'officer_of'], 
                        officers, 
                        left_on = 'node_1', 
                        right_on = 'node_id', 
                        how='right')

final_merged = pd.merge(edges_merged, 
                        entities, 
                        left_on = 'node_2', 
                        right_on = 'node_id', 
                        how='left')

final_merged.columns = final_merged.columns.str.replace('_y', '_entity')
final_merged.columns = final_merged.columns.str.replace('_x', '_officer')

### Limit columns

In [4]:
final_merged = final_merged[['name_officer',
                             'name_entity',
                             'countries_officer',
                             'address_officer',
                             'countries_entity',
                             'address_entity']]

### How many unique companies and people?

In [12]:
total = final_merged.shape[0]
companies = len(final_merged['name_entity'].unique())
people = len(final_merged['name_officer'].unique())

print("Number unique companies: {}, number of unique people: {}".format(companies, people))

Number unique companies: 101743, number of unique people: 185971


In [13]:
final_merged.drop_duplicates(inplace=True)
final_merged['address_officer'] = final_merged['address_entity'].astype('str')
final_merged['address_officer'] = final_merged['address_entity'].str.replace('nan', '0')
final_merged = final_merged[final_merged['name_entity'] != 'ORIFOUND DEVELOPMENT LIMITED']
for column in final_merged.columns:
    final_merged[column] = final_merged[column].str.replace('|', '')

In [14]:
final_merged.dtypes

name_officer         object
name_entity          object
countries_officer    object
address_officer      object
countries_entity     object
address_entity       object
dtype: object

In [15]:
final_merged.to_csv('panama/panama_papers_cleaned.csv', sep="|", index=False, na_rep='0')